In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

# Processing

In [46]:
# Load the dataset
df_train = pd.read_csv('../datasets/new_train.csv', index_col=0)

In [47]:
train = df_train.copy()

listBinaryLabels=len(list(train['label'].values))*[0]
for index,row in train.iterrows():
    if row['label']==2:
        listBinaryLabels[index]=2
        
train['binary_label']=np.array(listBinaryLabels)

# Training

In [48]:
# we build the df for binary classification and random forest
train_binary=train.drop('label',axis=1)
train_forest=train[train['binary_label']==0].drop('binary_label',axis=1)

# train_binary.head(10)
train_forest.head(10)

,org,tld,ccs,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,label,day,month,hour
Id,,,,,,,,,,,,,,
0,coursera,org,0,multipart/alternative,23,188,0,1,38,136818,0,Thu,Mar,1.950000
1,google,com,0,multipart/alternative,1,6,0,0,44,2467,0,Fri,Jan,5.333333
3,na,na,0,multipart/alternative,4,43,0,0,61,13775,0,Fri,Mar,3.766667
4,linkedin,com,0,multipart/alternative,4,26,0,0,29,22601,1,Thu,Jun,3.850000
6,flipkartletters,com,0,text/html,42,97,1,0,88,68612,3,Mon,Jan,2.466667
7,amazon,com,0,multipart/alternative,9,74,1,1,82,41354,0,Tue,Jan,3.333333
8,classroom,google.com,0,multipart/alternative,4,10,0,0,101,4294,0,Thu,Aug,11.750000
10,nrsc,gov.in,0,multipart/alternative,0,4,1,0,32,1210,0,Fri,Apr,15.533333
11,google,com,0,text/html,4,20,0,0,35,7156,0,Sat,Apr,3.450000


## Binary classification

In [49]:
train_binary_encoded=train_binary.copy()
#Removing anoying features like day and month
train_binary_encoded.drop(['day','month'],axis=1,inplace=True)

#Seperating categorical values from others
train_binary_cat = train_binary_encoded[['org','tld','mail_type']]
train_binary_num = train_binary_encoded.drop(['org','tld','mail_type','binary_label'],axis=1)

#Saving the labels somewhere
y_train_binary=train_binary_encoded['binary_label'].values

#Switching to arrays
X_train_binary_cat = train_binary_cat.values

encoder=OneHotEncoder(handle_unknown='ignore',sparse=False)
X_train_binary_encoded = encoder.fit_transform(X_train_binary_cat)

X_train_binary_fully_encoded=np.concatenate((X_train_binary_encoded,train_binary_num.values),axis=1)

In [50]:
# Standardize features
sc_binary = StandardScaler()
X_train_binary = sc_binary.fit_transform(X_train_binary_fully_encoded)

In [51]:
# Fit Gaussian Naive Bayes to the Training set
classifierNB = GaussianNB()
classifierNB.fit(X_train_binary, y_train_binary);

## Random Forest

In [52]:
train_forest_encoded=train_forest.copy()
#Removing anoying features like day and month
train_forest_encoded.drop(['day','month'],axis=1,inplace=True)

#Seperating categorical values from others
train_forest_cat = train_forest_encoded[['org','tld','mail_type']]
train_forest_num = train_forest_encoded.drop(['org','tld','mail_type','label'],axis=1)

#Saving the labels somewhere
y_train_forest=train_forest_encoded['label'].values

#Switching to arrays
X_train_forest_cat = train_forest_cat.values

X_train_forest_encoded = encoder.transform(X_train_forest_cat)

X_train_forest_fully_encoded=np.concatenate((X_train_forest_encoded,train_forest_num.values),axis=1)

In [53]:
# Standardize features
sc_forest = StandardScaler()
X_train_forest = sc_forest.fit_transform(X_train_forest_fully_encoded)

In [54]:
# Fit Random Forest to the Training set
classifierRF = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifierRF.fit(X_train_forest, y_train_forest);

# Prediction
## Binary classification

In [55]:
test_binary_encoded=df_test.copy()
#Removing anoying features like day and month
test_binary_encoded.drop(['day','month'],axis=1,inplace=True)

#Seperating categorical values from others
test_binary_cat = test_binary_encoded[['org','tld','mail_type']]
test_binary_num = test_binary_encoded.drop(['org','tld','mail_type'],axis=1)

#Switching to arrays
X_test_binary_cat = test_binary_cat.values

X_test_binary_encoded = encoder.transform(X_test_binary_cat)

X_test_binary_fully_encoded=np.concatenate((X_test_binary_encoded,test_binary_num.values),axis=1)

In [56]:
# Standardize features
X_test_binary = sc_binary.transform(X_test_binary_fully_encoded)

In [57]:
y_binary_labels = classifierNB.predict(X_test_binary)
df_test['binary_label']=y_binary_labels
df_test.head(4)

,org,tld,ccs,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,day,month,hour,binary_label
Id,,,,,,,,,,,,,,
0,iiitd,ac.in,4,multipart/alternative,1,15,1,0,50,9733,Sun,Aug,0.950000,2
1,usief,org.in,0,multipart/alternative,0,45,0,0,50,127388,Tue,Oct,1.983333,0
2,entropay,com,0,multipart/alternative,1,6,1,0,40,4003,Tue,Nov,12.033333,0
3,crm,jabong.com,0,text/html,13,46,0,0,24,21938,Mon,Jun,9.300000,0


## Random forest

In [58]:
test_forest_encoded=df_test[df_test['binary_label']==0]
#Removing anoying features like day and month
test_forest_encoded.drop(['day','month'],axis=1,inplace=True)

#Seperating categorical values from others
test_forest_cat = test_forest_encoded[['org','tld','mail_type']]
test_forest_num = test_forest_encoded.drop(['org','tld','mail_type','binary_label'],axis=1)

#Switching to arrays
X_test_forest_cat = test_forest_cat.values

X_test_forest_encoded = encoder.transform(X_test_forest_cat)

X_test_forest_fully_encoded=np.concatenate((X_test_forest_encoded,test_forest_num.values),axis=1)

D:\Programmes files (x86)\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [59]:
# Standardize features
X_test_forest = sc_forest.transform(X_test_forest_fully_encoded)

In [60]:
y_forest_labels = classifierRF.predict(X_test_forest)

res=[]
indice=0

for index,row in df_test.iterrows():
    if row['binary_label']==2:
        res.append(2)
    else:
        res.append(y_forest_labels[indice])
        indice+=1

df_test['label']=np.array(res)
df_test.head(25)

,org,tld,ccs,mail_type,images,urls,salutations,designation,chars_in_subject,chars_in_body,day,month,hour,binary_label,label
Id,,,,,,,,,,,,,,,
0,iiitd,ac.in,4,multipart/alternative,1,15,1,0,50,9733,Sun,Aug,0.950000,2,2
1,usief,org.in,0,multipart/alternative,0,45,0,0,50,127388,Tue,Oct,1.983333,0,0
2,entropay,com,0,multipart/alternative,1,6,1,0,40,4003,Tue,Nov,12.033333,0,0
3,crm,jabong.com,0,text/html,13,46,0,0,24,21938,Mon,Jun,9.300000,0,3
4,hackerrank,com,0,multipart/alternative,3,30,0,0,40,16854,Sun,May,12.916667,0,0
5,codechef,com,0,multipart/alternative,7,35,1,0,45,21688,Sat,Nov,18.316667,0,0
6,kaggle,com,0,text/plain,0,2,1,0,23,665,Tue,May,2.533333,0,0
7,iiitd,ac.in,0,multipart/alternative,0,2,0,0,70,2445,Tue,Oct,14.300000,2,2
8,iiitd,ac.in,1,multipart/alternative,0,3,1,0,36,9498,Wed,Dec,19.966667,2,2


# Saving the results

In [61]:
# Save results to submission file
y_pred = pd.DataFrame(res, columns=['label'])
y_pred.to_csv("../datasets/binary_classification_forest2.csv", index=True, index_label='Id')